# Transfer learning Part 1

In this notebook we'll learn how to import pre-trained model and use them for our problem (in this part we'll freeze the layers and only add an output layer)

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download gpiosenka/100-bird-species

100% 1.95G/1.96G [00:12<00:00, 226MB/s]
100% 1.96G/1.96G [00:12<00:00, 172MB/s]


In [7]:
! unzip /content/100-bird-species.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/WHITE TAILED TROPIC/120.jpg  
  inflating: train/WHITE TAILED TROPIC/121.jpg  
  inflating: train/WHITE TAILED TROPIC/122.jpg  
  inflating: train/WHITE TAILED TROPIC/123.jpg  
  inflating: train/WHITE TAILED TROPIC/124.jpg  
  inflating: train/WHITE TAILED TROPIC/125.jpg  
  inflating: train/WHITE TAILED TROPIC/126.jpg  
  inflating: train/WHITE TAILED TROPIC/127.jpg  
  inflating: train/WHITE TAILED TROPIC/128.jpg  
  inflating: train/WHITE TAILED TROPIC/129.jpg  
  inflating: train/WHITE TAILED TROPIC/130.jpg  
  inflating: train/WHITE TAILED TROPIC/131.jpg  
  inflating: train/WHITE TAILED TROPIC/132.jpg  
  inflating: train/WHITE TAILED TROPIC/133.jpg  
  inflating: train/WHITE TAILED TROPIC/134.jpg  
  inflating: train/WHITE TAILED TROPIC/135.jpg  
  inflating: train/WHITE TAILED TROPIC/136.jpg  
  inflating: train/WHITE TAILED TROPIC/137.jpg  
  inflating: train/WHITE TAILED TROPIC/138.jpg  
  inflating: train

In [15]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [9]:
target_size = (224, 224)

# creating an image data generator object and doing some data augmuntation on training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, #scaling each pixel to 0 to 1
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True)

# now connecting the object we just created above to train directory
train_generator = train_datagen.flow_from_directory('/content/train', # traingset path directory
                                                    target_size=target_size, # images will become 224x224
                                                    batch_size=32,
                                                    class_mode='categorical') # because target is categorical "100 different categories"

# creating another image data generator object, we dont use data augumnetation on val set!
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# connecting it to test directory
val_generator = val_datagen.flow_from_directory('/content/valid',
                                                   target_size=target_size,
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [10]:
class_indices = train_generator.class_indices

# Print class indices and labels
for class_index, class_label in class_indices.items():
    print(f'Class Index: {class_index}, Class Label: {class_label}')

Class Index: ABBOTTS BABBLER, Class Label: 0
Class Index: ABBOTTS BOOBY, Class Label: 1
Class Index: ABYSSINIAN GROUND HORNBILL, Class Label: 2
Class Index: AFRICAN CROWNED CRANE, Class Label: 3
Class Index: AFRICAN EMERALD CUCKOO, Class Label: 4
Class Index: AFRICAN FIREFINCH, Class Label: 5
Class Index: AFRICAN OYSTER CATCHER, Class Label: 6
Class Index: AFRICAN PIED HORNBILL, Class Label: 7
Class Index: AFRICAN PYGMY GOOSE, Class Label: 8
Class Index: ALBATROSS, Class Label: 9
Class Index: ALBERTS TOWHEE, Class Label: 10
Class Index: ALEXANDRINE PARAKEET, Class Label: 11
Class Index: ALPINE CHOUGH, Class Label: 12
Class Index: ALTAMIRA YELLOWTHROAT, Class Label: 13
Class Index: AMERICAN AVOCET, Class Label: 14
Class Index: AMERICAN BITTERN, Class Label: 15
Class Index: AMERICAN COOT, Class Label: 16
Class Index: AMERICAN DIPPER, Class Label: 17
Class Index: AMERICAN FLAMINGO, Class Label: 18
Class Index: AMERICAN GOLDFINCH, Class Label: 19
Class Index: AMERICAN KESTREL, Class Label:

In [11]:
class_indices

{'ABBOTTS BABBLER': 0,
 'ABBOTTS BOOBY': 1,
 'ABYSSINIAN GROUND HORNBILL': 2,
 'AFRICAN CROWNED CRANE': 3,
 'AFRICAN EMERALD CUCKOO': 4,
 'AFRICAN FIREFINCH': 5,
 'AFRICAN OYSTER CATCHER': 6,
 'AFRICAN PIED HORNBILL': 7,
 'AFRICAN PYGMY GOOSE': 8,
 'ALBATROSS': 9,
 'ALBERTS TOWHEE': 10,
 'ALEXANDRINE PARAKEET': 11,
 'ALPINE CHOUGH': 12,
 'ALTAMIRA YELLOWTHROAT': 13,
 'AMERICAN AVOCET': 14,
 'AMERICAN BITTERN': 15,
 'AMERICAN COOT': 16,
 'AMERICAN DIPPER': 17,
 'AMERICAN FLAMINGO': 18,
 'AMERICAN GOLDFINCH': 19,
 'AMERICAN KESTREL': 20,
 'AMERICAN PIPIT': 21,
 'AMERICAN REDSTART': 22,
 'AMERICAN ROBIN': 23,
 'AMERICAN WIGEON': 24,
 'AMETHYST WOODSTAR': 25,
 'ANDEAN GOOSE': 26,
 'ANDEAN LAPWING': 27,
 'ANDEAN SISKIN': 28,
 'ANHINGA': 29,
 'ANIANIAU': 30,
 'ANNAS HUMMINGBIRD': 31,
 'ANTBIRD': 32,
 'ANTILLEAN EUPHONIA': 33,
 'APAPANE': 34,
 'APOSTLEBIRD': 35,
 'ARARIPE MANAKIN': 36,
 'ASHY STORM PETREL': 37,
 'ASHY THRUSHBIRD': 38,
 'ASIAN CRESTED IBIS': 39,
 'ASIAN DOLLARD BIRD': 40,
 'AS

## Importing model

we use kaggle site to get pre-trained models and we also add a output layer

In [12]:
eff_model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b3-classification/versions/2"),
    tf.keras.layers.Dense(525, activation='softmax')
])
eff_model.build([None, 224, 224, 3])

In [13]:
eff_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                  metrics=['accuracy'])

In [14]:
eff_model.fit(train_generator, validation_data=val_generator, epochs=5)

Epoch 1/5
2645/2645 [==============================] - 1093s 407ms/step - loss: 2.3036 - accuracy: 0.6160 - val_loss: 1.1489 - val_accuracy: 0.7970
Epoch 2/5
2645/2645 [==============================] - 1071s 405ms/step - loss: 1.7329 - accuracy: 0.7505 - val_loss: 1.2293 - val_accuracy: 0.8076
Epoch 3/5
2645/2645 [==============================] - 1045s 395ms/step - loss: 1.5606 - accuracy: 0.7871 - val_loss: 1.2267 - val_accuracy: 0.8331
Epoch 4/5
2645/2645 [==============================] - 1053s 398ms/step - loss: 1.4662 - accuracy: 0.8101 - val_loss: 1.1461 - val_accuracy: 0.8488
Epoch 5/5
2645/2645 [==============================] - 1050s 397ms/step - loss: 1.3573 - accuracy: 0.8281 - val_loss: 1.1763 - val_accuracy: 0.8552


In [16]:
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
# Choose 9 random images from test dataset
categories = os.listdir('/content/train')
chosen_images = []
for category in random.sample(categories, 9):
    category_path = os.path.join("/content/train", category)
    image_name = random.choice(os.listdir(category_path))
    image_path = os.path.join(category_path, image_name)
    chosen_images.append((image_path, category))

# Predict and visualize
fig, axes = plt.subplots(3, 3, figsize=(10, 10))

for (image_path, true_label), ax in zip(chosen_images, axes.flatten()):
    img_array = load_and_preprocess_image(image_path)
    prediction = eff_model.predict(img_array)
    predicted_label_index = np.argmax(prediction)
    predicted_label = [key for key, value in class_indices.items() if value == predicted_label_index][0]  # Mapping index to class name

    ax.imshow(tf.keras.preprocessing.image.load_img(image_path))
    ax.axis('off')
    if predicted_label_index == true_label:
        ax.set_title('Correct Prediction: {}'.format(predicted_label), color='green')
    else:
        ax.set_title('Incorrect Prediction: {}'.format(predicted_label), color='red')

plt.tight_layout()
plt.show()